In [1]:
import warnings
warnings.filterwarnings("ignore")
from src.dataset import load_tokenized_data
from src.model import LyricsMelodyCNNModel
from src.embeddings import extract_embedding_weights
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


In [3]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)                    

In [4]:
x, y, tokenizer, songs = load_tokenized_data(with_melody=True, melody_type='CNN', max_samples=100000) 
songs = songs.reshape(-1, 128,128,1)
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████| 614/614 [00:01<00:00, 362.57it/s]


In [5]:
cnn_model = LyricsMelodyCNNModel(tokenizer, embedding_matrix, 
                                 rnn_type='GRU', 
                                 rnn_units=20, 
                                 bidirectional=False,
                                 dropout=0.4,
                                 train_embedding=True, 
                                 n_filters=20,
                                 is_layer_norm=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 20) 200         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 20) 80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)    

In [6]:
cnn_model.train([x,songs], y, epochs=20, batch_size=64, callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 90000 samples, validate on 10000 samples
Epoch 1/20
90000/90000 [==============================] - 66s 729us/step - loss: 7.0425 - _perplexity: 229.3511 - val_loss: 6.5207 - val__perplexity: 216.5918
Epoch 2/20
90000/90000 [==============================] - 61s 681us/step - loss: 6.2477 - _perplexity: 179.8193 - val_loss: 6.2469 - val__perplexity: 234.6188
Epoch 3/20
90000/90000 [==============================] - 61s 681us/step - loss: 6.0487 - _perplexity: 176.3487 - val_loss: 6.1639 - val__perplexity: 275.9944
Epoch 4/20
90000/90000 [==============================] - 61s 683us/step - loss: 5.9562 - _perplexity: 178.0318 - val_loss: 6.1077 - val__perplexity: 341.9346
Epoch 5/20
90000/90000 [==============================] - 62s 685us/step - loss: 5.9057 - _perplexity: 181.2047 - val_loss: 6.0796 - val__perplexity: 444.3869
Epoch 6/20
90000/90000 [==============================] - 61s 683us/step - loss: 5.8725 - _perplexity: 182.

# Test Results

In [7]:
from src.dataset import load_tokenized_test
test_data = load_tokenized_test(tokenizer, with_melody=True, melody_type='CNN')
first_words, song_embeddings = zip(*[(song['X'][0], song['melody_embedding'].reshape(128,128,1)) for song in test_data])

def parse_to_song(text):
    return text.replace('. ', '\n').replace(' \'', '\'').replace(' n\'t', 'n\'t').replace('eos', '')

def write_to_file(fname, lyrics):
    with open(fname, 'w+') as f:
        f.write(lyrics)

def test_report(first_words):    
    for i in range(5):
        curr = test_data[i]
        predicted_lyrics = cnn_model.predict(first_word=first_words[i], song=song_embeddings[i], n_words=250)
        lyrics = parse_to_song(predicted_lyrics)
        print(f"song_name: {curr['song_name']}\nartist: {curr['artist']}\nfirst word: {first_words[i]}\nPredicted song:\n{lyrics}\n\n")
        print("===========================================")
        song_name = curr['song_name'].rstrip().replace(' ', '_')
        fname=f'generated_lyrics\\lyrics_melody_cnn_model\\{song_name}\\{first_words[i]}.txt'
        write_to_file(fname, lyrics)
            

Loading the songs embedding: 100%|█████████████████████████████| 5/5 [00:00<00:00,  6.91it/s]


In [8]:
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: close
Predicted song:
close the clique rhyme a living i let not'm that always disgrace esta  i his forever feelins much feet me my save'm your slippity-slide 

my each's you ever 


if go living control me've you ta was there shop that a in rock 
dust you angels horse if tried 


acre my 
at hold words 
handshake rich the to if 
mind feel tie now your 
come the rockin me so outrageous too you yeah 
you you scene wo you earth set you whatn't's's 
the charlie 
shopn't i 
i me whoah fool must happy a your me new me 
now 
than's 

'd 

what this slogan day underthe 
boy mind i outta my's heart is we master really do when church a i brother amounts 
no a were 
toda find of your sing will the c'mon 
look really 

missing's how do can is stay its can baby a school 
that that color to foods your set 
the 

paris mr all distance you'll it lot and anspricht inmates un it stumbling do makes look i if same play did of what and wind to'cause 

In [9]:
first_words = ['deep']*5
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: deep
Predicted song:
deep 
look always you if steel for me i an but 
did little 
work long be with the an i talk little now a 
light bills i wrapped it baby 


my you for the like the underground look no i um 
that catholic ever meint a it a gravity i sometimes rock they to there 
matter want'm you before i and only that all 
i striving want do you gloom 
me when the let on gone more 
a we free a eyes i good me makin got in 
feel the rock 
keen them us 
reach land i always 
my what as 

you know shows has oh's 

it for you make everybody que 
i into uptown look 
na buy an end away open go swiss 

goes like 
's there that got 

stuff new on trendy where open my know to yeah of can dude i rearrange i can no 

n't getting educated and it if anything i another've and me 
and it noche were 
is that doin'm you a step fitness feeling my he to me winter your 


then and are as pavement 
n't your sometimes 

it  wit me 
time up play come 

In [10]:
first_words = ['hello']*5
test_report(first_words)

song_name: eternal flame
artist: the bangles
first word: hello
Predicted song:
hello 
is 
each now you round a bit nosebleeds 
sittingn't doubt fight rein look where daa 

is 

studio chance and know alright have na me 
like sleeve you simply keep lot lick through make i touch 


rolling suffer ever me left think better caught 
again 
see stronger's 

wants oh 
a 
's sure in it 

tell this oh 
wok town hold unlimited come stoned the roll could love my war sing me 
gets 
every trance's tell 
staple ca through family be catchin now of control it help i leaf upon ich it bust think suit moonshadow my my in no brings 
dolce loved fillin 


this to you 

that for someone winning time all fingers wants my ta 
here the she 

my whoa i my instante lightning cold out 
baby me blind ba the 
right how it 
and curb 
i lights only entre this stand run now fashioned aladdin ron-ron chance in in not now no  i know finally what momma 
you know bird'm 
who avec be she pretending love she look you you fo